In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Skrypt do łączenia plików CSV dla każdej strony ze wszystkich lat
"""

import pandas as pd
import os
from pathlib import Path

# Lista stron
ALLOWED_SITES = [
    "forsal.pl",
    "businessinsider.com.pl",
    "pap.pl",
    "tvn24.pl",
    "obserwatorfinansowy.pl",
    "pie.net.pl"
]

# Lata
YEARS = [2021, 2022, 2023]

# Folder źródłowy i wyjściowy
SOURCE_FOLDER = "csv ze stron"
OUTPUT_FOLDER = "csv ze stron"

def concatenate_site_csvs(site: str, years: list, source_folder: str = ".", output_folder: str = "."):
    """
    Łączy pliki CSV dla danej strony ze wszystkich lat.
    
    Args:
        site: nazwa strony (np. 'forsal.pl')
        years: lista lat do połączenia
        source_folder: folder gdzie znajdują się pliki CSV
        output_folder: folder gdzie zapisać wynikowy plik
    """
    site_normalized = site.replace('.', '_')
    dataframes = []
    
    print(f"\n[INFO] Łączenie plików dla {site}...")
    
    for year in years:
        filename = os.path.join(source_folder, f"{site_normalized}_{year}.csv")
        
        if os.path.exists(filename):
            try:
                df = pd.read_csv(filename)
                # Dodaj kolumnę z rokiem dla łatwiejszej analizy
                df['year'] = year
                dataframes.append(df)
                print(f"[INFO]  ✓ Wczytano {filename}: {len(df)} rekordów")
            except Exception as e:
                print(f"[ERROR]  ✗ Błąd przy wczytywaniu {filename}: {e}")
        else:
            print(f"[WARN]  ⚠ Plik {filename} nie istnieje")
    
    if not dataframes:
        print(f"[WARN] Brak plików do połączenia dla {site}")
        return None
    
    # Połącz wszystkie dataframe'y
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    # Usuń duplikaty na podstawie URL
    initial_count = len(combined_df)
    combined_df = combined_df.drop_duplicates(subset=['url'], keep='first')
    duplicates_removed = initial_count - len(combined_df)
    
    if duplicates_removed > 0:
        print(f"[INFO]  Usunięto {duplicates_removed} duplikatów")
    
    # Sortuj według daty (jeśli istnieje) lub roku
    if 'google_detected_date' in combined_df.columns:
        combined_df = combined_df.sort_values('google_detected_date')
    else:
        combined_df = combined_df.sort_values('year')
    
    # Zapisz połączony plik jako full_{site}.csv
    output_filename = os.path.join(output_folder, f"full_{site_normalized}.csv")
    combined_df.to_csv(output_filename, index=False, encoding='utf-8')
    
    print(f"[INFO]  ✓ Zapisano {output_filename}: {len(combined_df)} rekordów")
    
    return combined_df

def main():
    # Utwórz folder wyjściowy jeśli nie istnieje
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    
    print("[INFO] 🚀 Rozpoczynam łączenie plików CSV...")
    print(f"[INFO] 📂 Pliki źródłowe: {SOURCE_FOLDER}")
    print(f"[INFO] 📁 Pliki wynikowe zostaną zapisane w: {OUTPUT_FOLDER}")
    
    total_files = 0
    total_records = 0
    
    # Połącz pliki dla każdej strony
    for site in ALLOWED_SITES:
        result = concatenate_site_csvs(site, YEARS, SOURCE_FOLDER, OUTPUT_FOLDER)
        if result is not None:
            total_files += 1
            total_records += len(result)
    
    print(f"\n[INFO] ✅ Zakończono łączenie plików!")
    print(f"[INFO] 📊 Utworzono {total_files} plików full_*.csv")
    print(f"[INFO] 📝 Łącznie {total_records} unikalnych artykułów")

if __name__ == "__main__":
    main()


[INFO] 🚀 Rozpoczynam łączenie plików CSV...
[INFO] 📂 Pliki źródłowe: csv ze stron
[INFO] 📁 Pliki wynikowe zostaną zapisane w: csv ze stron

[INFO] Łączenie plików dla forsal.pl...
[WARN]  ⚠ Plik csv ze stron/forsal_pl_2021.csv nie istnieje
[WARN]  ⚠ Plik csv ze stron/forsal_pl_2022.csv nie istnieje
[WARN]  ⚠ Plik csv ze stron/forsal_pl_2023.csv nie istnieje
[WARN] Brak plików do połączenia dla forsal.pl

[INFO] Łączenie plików dla businessinsider.com.pl...
[INFO]  ✓ Wczytano csv ze stron/businessinsider_com_pl_2021.csv: 100 rekordów
[INFO]  ✓ Wczytano csv ze stron/businessinsider_com_pl_2022.csv: 100 rekordów
[INFO]  ✓ Wczytano csv ze stron/businessinsider_com_pl_2023.csv: 100 rekordów
[INFO]  Usunięto 118 duplikatów
[INFO]  ✓ Zapisano csv ze stron/full_businessinsider_com_pl.csv: 182 rekordów

[INFO] Łączenie plików dla pap.pl...
[INFO]  ✓ Wczytano csv ze stron/pap_pl_2021.csv: 82 rekordów
[INFO]  ✓ Wczytano csv ze stron/pap_pl_2022.csv: 96 rekordów
[INFO]  ✓ Wczytano csv ze stron/pap

In [ ]:
money_df = 